In [1]:
!pip3 install ktrain


     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 6.8 MB 34.6 MB/s 
     |████████████████████████████████| 981 kB 47.0 MB/s 
     |████████████████████████████████| 263 kB 60.9 MB/s 
     |████████████████████████████████| 2.8 MB 28.5 MB/s 
     |████████████████████████████████| 1.2 MB 67.4 MB/s 
     |████████████████████████████████| 468 kB 56.5 MB/s 
     |████████████████████████████████| 596 kB 81.3 MB/s 
     |████████████████████████████████| 61 kB 461 kB/s 
     |████████████████████████████████| 895 kB 81.2 MB/s 
     |████████████████████████████████| 3.3 MB 74.0 MB/s 
  Created wheel for ktrain: filename=ktrain-0.28.3-py3-none-any.whl size=25292659 sha256=7e69ce4f262de8b270ced08763abf9863cf0dc862b4e737310e88f8d57ebb8f7
  Stored in directory: /root/.cache/pip/wheels/6a/7e/c3/f46cdfc2b81c54424923b1405d7e670c35cacc11ada9a47b1c
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=235e231d0880c277

In [2]:
!pip install autocorrect

     |████████████████████████████████| 622 kB 10.2 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622381 sha256=9876153375e9d8116fdc791073c0d45b4c6b382bbb66ad6dd54375cabcc1e942
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect


In [3]:
# import ktrain and the ktrain.text modules
import ktrain


In [4]:
ktrain.__version__

'0.28.3'

In [5]:
%matplotlib inline

# # Importar bibliotecas (pandas, numpy, CountVectorizer, TfidfVectorizer, TfidfTransformer, SGDClassifier, Pipeline)
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import nltk
from difflib import SequenceMatcher

from autocorrect import Speller
from sklearn.metrics.pairwise import cosine_similarity


In [6]:
spell = Speller( lang='pt')


dictionary for this language not found, downloading...
__________________________________________________
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [7]:
spell = Speller( lang='pt')
meu_texto = "testte"
meu_texto1 = spell(meu_texto)
print(meu_texto1)


deste


In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
%cd /content/drive/MyDrive/Colab Notebooks/projeto/

/content/drive/MyDrive/Colab Notebooks/projeto


In [11]:
# Carrega os dados
df = pd.read_csv('dados_agrupados.csv',  sep=',', low_memory=False,encoding='latin-1' )



In [12]:
df.head()

,Numero,Livro,Capitulo,Pergunta,Resposta,Target,Target_final,Target_final1,Target_final2,Target_final3
0,1,Pera,Generalidades,Qual é o centro de origem da pereira?,São citados três centros de origem da pereira:...,Pera,Frutas,NaN,Frutas,NaN
1,2,Pera,Generalidades,Qual é o centro de origem mais importante?,O centro do Oriente Médio é considerado de imp...,Pera,Frutas,NaN,Frutas,NaN
2,3,Pera,Generalidades,Como ocorreu a disseminação da pereira pelo mu...,"Com base em estudos bioquímicos, verificou-se ...",Pera,Frutas,NaN,Frutas,NaN
3,4,Pera,Generalidades,Quais são as espécies de pereira mais cultivad...,"Na Europa, na América do Norte, na América do ...",Pera,Frutas,NaN,Frutas,NaN
4,5,Pera,Generalidades,Quando a pereira foi introduzida no Brasil?,Não há relatos na literatura sobre a introduçã...,Pera,Frutas,NaN,Frutas,NaN


In [13]:
# Carrega biblioteca para salvar e carregar o modelo treinado (joblib)
import joblib
# Carrega modelo classificador do disco
modelo_classificador_geral = joblib.load('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/modelo_chatbot_p1.pkl')

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator SGDClassifier from version 1.0.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator Pipeline from version 1.0.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserW

In [14]:
modelo_classificador_cap_graos = ktrain.load_predictor('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/my_predictor_graos')


In [15]:

modelo_classificador_cap_frutas = ktrain.load_predictor('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/my_predictor_frutas')


In [16]:

modelo_classificador_cap_animais = ktrain.load_predictor('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/my_predictor_animais')


In [17]:

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [18]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [19]:
df['texto_sem_stopwords'] = df['Pergunta'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))


In [20]:
  temp = df[0:1]


In [21]:
  temp['Livro']


0    Pera
Name: Livro, dtype: object

In [22]:

def similarGeral(meu_texto, df_local):
  tfidf = TfidfVectorizer(min_df=1, max_df=.7, max_features=2000, stop_words=stopwords, ngram_range=(1,2))
  tfidf.fit(df_local['texto_sem_stopwords'])

  X_idf = tfidf.transform(df_local['texto_sem_stopwords'])

    
  meu_texto = str(spell(meu_texto))

  sim = cosine_similarity(X_idf, tfidf.transform( [meu_texto] ))
  df_local['sim'] = sim
  value = 0

  temp = df_local[0:1]
  valorsimilar = temp['sim']
  max_row = []

  for index, row in df_local.iterrows():
    #print(row['Pergunta'])
    valorsimilar = row['sim']
    if (valorsimilar > value):
        value = row['sim']
        max_row  = row
      #print(row['Pergunta'])

  #return df_local.iloc[df_local['sim'].idxmax()]
  return max_row

In [23]:
def similarGeralPropria(meu_texto, df):
  meu_texto = str(spell(meu_texto))

  value = 0

  max_row = []

  for index, row in df.iterrows():
    #print(row['Pergunta'])

    valorsimilar = similar( meu_texto, row['Pergunta'])
    if (valorsimilar > value):
      value = similar(meu_texto, row['Pergunta'])
      max_row  = row
      #print(row['Pergunta'])

  return max_row 

In [24]:
def my_stopwords(nova_pergunta):
  saida = ""
  for word in nova_pergunta.split():
    if (word not in stopwords): 
      saida = saida + " " + word
  
  return saida

In [48]:
def response(modelo_classificador, nova_pergunta, ponto_corte):
    #nova_pergunta = str(spell(nova_pergunta))
    print(nova_pergunta)
    nova_pergunta = nova_pergunta.lower()
    pergunta = [my_stopwords(nova_pergunta).strip()]
    print(pergunta)
    result = modelo_classificador.predict(pergunta)
    prob_preds= modelo_classificador_geral.predict_proba(pergunta)
  
    print("SGDClassifier")
    print(result[0])
    
    percentual = 1
    if (result[0] == 'Frutas'):
      df_cap = df.loc[df['Target_final'] == result[0]]
      percentual = prob_preds[0][1]
      #modelo_classificador_cap_local = joblib.load('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/modelo_chatbot_p2_frutas.pkl')
      modelo_classificador_cap_local = modelo_classificador_cap_frutas
    elif  (result[0] == 'Graos'):
      percentual = prob_preds[0][2]
      df_cap = df.loc[df['Target_final'] == result[0]]
      #modelo_classificador_cap_local = joblib.load('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/modelo_chatbot_p2_graos.pkl')
      modelo_classificador_cap_local = modelo_classificador_cap_graos

    elif  (result[0] == 'Animal'):
      percentual = prob_preds[0][0]
      df_cap = df.loc[df['Target_final'] == result[0]]
      #modelo_classificador_cap_local = joblib.load('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/modelo_chatbot_p2_animais.pkl')
      modelo_classificador_cap_local = modelo_classificador_cap_animais

    print(percentual)
    if (percentual > ponto_corte):
      print("BERT:")
      result_cap = modelo_classificador_cap_local.predict(pergunta)
      #print(result_cap)
      df_cap_final = df_cap.loc[(df_cap['Target'] == result_cap[0])]
      resposta = similarGeral(question,df_cap_final.copy())
  
    else:
      resposta = []
    
  
    return resposta

In [50]:
question = 'Qual o valor nutritivo?'
#question = 'O que é levedura?'
#question = 'Na botânica, qual a classificação da manga?'
print("TESTE: ")
answer = response(modelo_classificador_geral, question, 0.7)

#print("Sua pergunta: " + question)
#print(answer)
if (len(answer) > 0):
  print("ML: ")
  print("Categoria: "  + answer['Target_final'])
  print("Tema: "  + answer['Target'])
  print("Pergunta: "  + answer['Pergunta'])
  print("Resposta: "  + answer['Resposta'])
else:
  answerSim = similarGeral(question, df)
  if (len(answerSim) > 0):
    print("SIM cosine_similarity: ")
    print("Categoria: "  + answerSim.Target_final)
    print("Tema: "  + answerSim.Target)
    print("Pergunta: "  + answerSim.Pergunta)
    print("Resposta: "  + answerSim.Resposta)

  else:
    answerSimP = similarGeralPropria(question, df)
    print("sim SequenceMatcher: ")
    print("Categoria: "  + answerSimP.Target_final)
    print("Tema: "  + answerSimP.Target)
    print("Pergunta: "  + answerSimP.Pergunta)
    print("Resposta: "  + answerSimP.Resposta)


TESTE: 
Qual o valor nutritivo?
['valor nutritivo?']
SGDClassifier
Graos
0.5250966928829419
SIM cosine_similarity: 
Categoria: Graos
Tema: Milho
Pergunta: Qual o valor nutritivo da pipoca?
Resposta: Uma xícara (8 g) de pipoca estourada sem óleo e sal contém 31 kcal, 6,2 g de carboidratos (2% VD*), 1,2 g de fibras (5% VD), 1 g de proteína (1,3% VD), 0,3 g de óleo (0,5% VD), 1 mg de cálcio, 0,2 mg de ferro, 24 mg fósforo, 24 mg de potássio, 0,3 mg de sódio, 0,8 mg de zinco, 0,2 mg de tiamina, 0,3 mg de riboflavina, 2 mg de niacina, 15 UI de vitamina A e 0 mg de colesterol. Portanto, a pipoca é uma boa fonte de fibras e possui reduzido valor calórico, quando preparada sem óleo, manteiga ou gordura. *VD = valor diário relativo a uma dieta de 2.000 kcal.


In [ ]:
preds

In [ ]:
np.round(prob_preds[0:,0],6).tolist()

In [ ]:
np.round(prob_preds[0:,1],6).tolist

In [ ]:
np.round(prob_preds[0:,2],6).tolist()

In [ ]:
np.round(prob_preds[0:,3],6).tolist()


In [ ]:
df_prob['prob_pessoal(0)'] = np.round(prob_preds[0:,0],6).tolist()
df_prob['prob_licit(1)'] = np.round(prob_preds[0:,1],6).tolist()
df_prob['prob_respo(2)'] = np.round(prob_preds[0:,2],6).tolist()
df_prob['prob_dirproc(3)'] = np.round(prob_preds[0:,3],6).tolist()

NameError: ignored

In [ ]:

df.shape

(16596, 12)

In [ ]:
df

In [ ]:
answerSim

[]

In [ ]:
answer

,Numero,Livro,Capitulo,Pergunta,Resposta,Target,Target_final,Target_final1,Target_final2,Target_final3,texto_sem_stopwords,sim
5020,1,Caju,NaN,Qual é a origem do cajueiro?,O Brasil é o provável centro de origem do caju...,Caju,Frutas,NaN,Frutas,NaN,Qual origem cajueiro?,0.000000
5021,2,Caju,NaN,Qual é a distribuição geográfica do cajueiro?,O cajueiro encontra-se distribuído na maioria ...,Caju,Frutas,NaN,Frutas,NaN,Qual distribuição geográfica cajueiro?,0.000000
5022,3,Caju,NaN,Qual é a classificação taxonômica do cajueiro?,O cajueiro cultivado (Anacardium occidentale L...,Caju,Frutas,NaN,Frutas,NaN,Qual classificação taxonômica cajueiro?,0.000000
5023,4,Caju,NaN,Quais são as principais características botâni...,"O cajueiro é uma planta perene, com ramificaçã...",Caju,Frutas,NaN,Frutas,NaN,Quais principais características botânicas caj...,0.062574
5024,5,Caju,NaN,Qual é o número de cromossomos e o modo de rep...,O cajueiro é uma planta com predominância de p...,Caju,Frutas,NaN,Frutas,NaN,Qual número cromossomos modo reprodução cajueiro?,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
5523,504,Caju,NaN,Quais são os principais importadores mundiais ...,O mercado importador de ACC é formado por mais...,Caju,Frutas,NaN,Frutas,NaN,Quais principais importadores mundiais amêndoa...,0.034343
5524,505,Caju,NaN,Quais são os principais países exportadores de...,"Em 2010, as exportações mundiais de amêndoas d...",Caju,Frutas,NaN,Frutas,NaN,Quais principais países exportadores amêndoa c...,0.036702
5525,506,Caju,NaN,A Embrapa fornece/indica um apoio na etapa de ...,Não. Nossa indicação é que o exportador faça u...,Caju,Frutas,NaN,Frutas,NaN,A Embrapa fornece/indica apoio etapa exportaçã...,0.000000
5526,507,Caju,NaN,Mais alguma pergunta?,Caso tenha mais alguma pergunta preencha nosso...,Caju,Frutas,NaN,Frutas,NaN,Mais alguma pergunta?,0.000000


In [ ]:
tfidf = TfidfVectorizer(min_df=1, max_df=.7, max_features=2000, stop_words=stopwords, ngram_range=(1,2))
tfidf.fit(df['texto_sem_stopwords'])

X_idf = tfidf.transform(df['texto_sem_stopwords'])


In [ ]:
texto = 'Qual centro origem pere'
sim = cosine_similarity(X_idf, tfidf.transform( [texto] ))

In [ ]:
df['sim'] = sim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df['sim'].idxmax()

1568

In [ ]:
df.iloc[df['sim'].idxmax()]

Pergunta               O que é o centro de origem e o centro de domes...
Target_final                                                       Graos
texto_sem_stopwords         O centro origem centro domesticação espécie?
sim                                                             0.882173
Name: 1568, dtype: object

In [ ]:
df

,Pergunta,Target_final,texto_sem_stopwords,sim
0,Qual é o centro de origem da pereira?,Frutas,Qual centro origem pereira?,0.867505
1,Qual é o centro de origem mais importante?,Frutas,Qual centro origem importante?,0.866035
2,Como ocorreu a disseminação da pereira pelo mu...,Frutas,Como ocorreu disseminação pereira mundo?,0.000000
3,Quais são as espécies de pereira mais cultivad...,Frutas,Quais espécies pereira cultivadas mundo?,0.000000
4,Quando a pereira foi introduzida no Brasil?,Frutas,Quando pereira introduzida Brasil?,0.000000
...,...,...,...,...
16591,Qual é a produção nacional e a mundial de vinhos?,Frutas,Qual produção nacional mundial vinhos?,0.000000
16592,Quais são os maiores produtores e os maiores e...,Frutas,Quais maiores produtores maiores exportadores ...,0.000000
16593,Que quantidade de vinho o Brasil importa anual...,Frutas,Que quantidade vinho Brasil importa anualmente?,0.000000
16594,Qual é o consumo per capita dos países que mai...,Frutas,Qual consumo per capita países consomem vinhos?,0.000000


In [ ]:
while True:
    input_data = input("You- ")
    question = input_data

    answer = response(modelo_classificador_geral, question)
    print("Sua pergunta: " + question)
    print("Categoria: "  + answer['Target_final'])
    print("Livro: "  + answer['Livro'])
    print("Pergunta predita: "  + answer['Pergunta'])
    print("Resposta: "  + answer['Resposta'])
    print("__________________________________________________________________")
    print("Usando similaridade sem ML: ")
    print(similarGeral(question,df))
    print("__________________________________________________________________")

KeyboardInterrupt: ignored

Abaixo sao experimentos


In [ ]:
resposta = similarGeral(question,df_cap_final)
print(question)
print(answer1)
print (resposta['Pergunta'])
print (resposta['Resposta'])

NameError: ignored

In [ ]:
answer1 = response(modelo_classificador_geral, question)
print(answer1)

Frutas
[]


In [ ]:
answer1

[]

In [ ]:
if (answer1['Target_final'] == 'Frutas'):
  df_cap = df.loc[df['Target_final'] == answer1['Target_final']]
  modelo_classificador_cap = joblib.load('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/modelo_chatbot_p2_frutas.pkl')
elif  (answer1['Target_final'] == 'Graos'):
  df_cap = df.loc[df['Target_final'] == answer1['Target_final']]
  modelo_classificador_cap = joblib.load('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/modelo_chatbot_p2_graos.pkl')
elif  (result[0] == 'Animal'):
  df_cap = df.loc[df['Target_final'] == result[0]]
  modelo_classificador_cap = joblib.load('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/modelo_chatbot_p2_animais.pkl')



 

TypeError: ignored

In [ ]:
modelo_classificador_cap1

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', SGDClassifier(loss='log'))])

In [ ]:
df_cap = df_cap.dropna(axis=0, subset=['Capitulo'])


In [ ]:
pergunta = [question]
res = modelo_classificador_cap.predict(pergunta)

In [ ]:
res

array(['Colheita, Pós-colheita e Processamento'], dtype='<U57')

In [ ]:
answer2 = response(modelo_classificador_geral, question)
print(answer2)

Numero                                                         302
Livro                                                         Pera
Capitulo                                          Manejo de Pragas
Pergunta         Quais são os principais danos ocasionados por ...
Resposta         O ataque de ácaros diminui a capacidade fotoss...
Target                                                        Pera
Target_final                                                Frutas
Target_final1                                                  NaN
Target_final2                                               Frutas
Target_final3                                                  NaN
Name: 301, dtype: object


In [ ]:
question

'Quais são os danos causados nas videiras pela ocorrência de granizo?'

In [ ]:
pergunta = [question]
prob_preds= modelo_classificador_cap.predict_proba(pergunta)
prob_preds[:2]

array([[0.00455621, 0.00615948, 0.05931911, 0.00428504, 0.04699199,
        0.48640411, 0.00748867, 0.0047202 , 0.00552178, 0.00789402,
        0.01116788, 0.0287851 , 0.00475931, 0.00916698, 0.01311982,
        0.02657443, 0.02302615, 0.00741005, 0.00802682, 0.00606833,
        0.00648155, 0.00541293, 0.01691726, 0.00466449, 0.01897141,
        0.00834474, 0.00864936, 0.02271939, 0.01853888, 0.00921103,
        0.00373007, 0.00731142, 0.0047702 , 0.00556334, 0.01062779,
        0.00554999, 0.05245074, 0.0113669 , 0.00727303]])

In [ ]:
prob_preds[:2][0][5]

0.6709617290667808

In [ ]:
prob_preds[:2][0][27]

0.21694193419206076

In [ ]:
cap1 = modelo_classificador_cap.classes_[5]
cap1

'Colheita, Pós-colheita e Processamento'

In [ ]:
cap2 = modelo_classificador_cap.classes_[27]
cap2

'Produção, Mercado e Comercialização'

In [ ]:
  df_cap_final = df_cap.loc[(df_cap['Capitulo'] == cap1) | (df_cap['Capitulo'] == cap2)]


In [ ]:
df_cap_final.Capitulo.unique()

array(['Colheita, Pós-colheita e Processamento',
       'Produção, Mercado e Comercialização'], dtype=object)

In [ ]:
df_cap_final.head()

,Numero,Livro,Capitulo,Pergunta,Resposta,Target,Target_final,Target_final1,Target_final2,Target_final3
402,403,Pera,"Colheita, Pós-colheita e Processamento",O cloridrato de aviglicina pode ser utilizado ...,"Quando registrado para a cultura, o cloridrato...",Pera,Frutas,NaN,Frutas,NaN
403,404,Pera,"Colheita, Pós-colheita e Processamento",O cloridrato de aviglicina provoca algum efeit...,Como o cloridrato de aviglicina retarda a matu...,Pera,Frutas,NaN,Frutas,NaN
404,405,Pera,"Colheita, Pós-colheita e Processamento",Qual é o estádio de maturação ideal para a col...,As peras europeias não desenvolvem boa qualida...,Pera,Frutas,NaN,Frutas,NaN
405,406,Pera,"Colheita, Pós-colheita e Processamento",Como é avaliado o estádio de maturação para in...,A maturação para colheita está correlacionada ...,Pera,Frutas,NaN,Frutas,NaN
406,407,Pera,"Colheita, Pós-colheita e Processamento",Como é feita a colheita das peras?,"A colheita das peras é realizada manualmente, ...",Pera,Frutas,NaN,Frutas,NaN


In [ ]:
df_respostas = df_cap.loc[df_cap['Capitulo'] == answer2]

In [ ]:

resposta = similarGeral(question,df_cap_final)
print(question)
print(answer1)
print (resposta['Pergunta'])
print (resposta['Resposta'])

Como eu colho peras?
Frutas
Como é feita a colheita das peras?
A colheita das peras é realizada manualmente, por meio de uma ligeira elevação ou torção do fruto, para que haja o destacamento do pedúnculo na camada de abscisão. A manutenção do pedúnculo confere melhor conservação pós-colheita às peras; entretanto, o manuseio excessivo dos frutos


In [ ]:
answer = response(modelo_classificador_geral, question)
print(answer)

Numero                                                         407
Livro                                                         Pera
Capitulo                    Colheita, Pós-colheita e Processamento
Pergunta                        Como é feita a colheita das peras?
Resposta         A colheita das peras é realizada manualmente, ...
Target                                                        Pera
Target_final                                                Frutas
Target_final1                                                  NaN
Target_final2                                               Frutas
Target_final3                                                  NaN
Name: 406, dtype: object


In [ ]:
similar('Como eu colho peras?','Como é feita a colheita das peras?')

0.6666666666666666

In [ ]:
similar('Como eu colho peras?', 'Qual o montante de capital associado com investimentos e despesas operacionais para implantar e formar 1 ha de pera?')

0.23529411764705882

In [ ]:
question = 'Como o clima pode afetar um pomar altitude pereira?'
#Como as mudanças climáticas podem afetar um pomar localizado em regiões com altitude superior a 1.000 m?


In [ ]:
answer = response(modelo_classificador_geral, question)
print(question)
print("Respondendo a pergunta:"  + " " + answer['Pergunta'])
print(answer['Resposta'])

Frutas
Como o clima pode afetar um pomar altitude pereira?
Respondendo a pergunta: Como as mudanças climáticas podem afetar o cultivo da pereira?
As medições que vêm sendo feitas nos últimos anos, no Sul do Brasil, nas estações meteorológicas indicam um aumento bem maior das temperaturas mínimas do ar (cerca de 1,5 °C), o que indica que os invernos, na média, estão ficando com temperaturas mais elevadas e está havendo uma redução do número de horas de frio no outono e no inverno, período de repouso hibernal da pereira. Além disso, tem ocorrido aumento da variabilidade climática, com a ocorrência de eventos climáticos extremos. Embora tenham ocorrido, na média, anos mais quentes, alguns anos têm se apresentado bastante frios. Assim, a escolha da cultivar que melhor se adapte a cada local é


In [ ]:
while True:
    input_data = input("You- ")
    question = input_data
    answer = response(modelo_classificador_geral, question)
    #print(answer)
    print("Respondendo a pergunta:"  + " " + answer['Pergunta'])
    print(answer['Resposta'])

You- Como se faz o armazenamento de peras?
Respondendo a pergunta: Como se faz o armazenamento de peras?
As peras podem ser armazenadas no período compreendido entre a colheita e a indução do amadurecimento, seja por temperatura, seja por etileno. A refrigeração (-1,1 ºC a 0 ºC e umidade relativa de 90% a 95%) é a principal ferramenta para a preservação da qualidade e o prolongamento da vida útil das peras. Dependendo da qualidade, as peras podem ser armazenadas sob condições atmosféricas normais ou sob atmosfera controlada. A atmosfera modificada, associada à refrigeração, pode estender o período de armazenamento de peras, principalmente para os pequenos produtores que não dispõem de sistema de armazenamento sob atmosfera controlada.
You- É utilizada cera no beneficiamento de peras?
Respondendo a pergunta: É utilizada cera no beneficiamento de peras?
Não. Nenhum tipo de cera é utilizado no beneficiamento de peras.
You- Como é feita a colheita das peras?
Respondendo a pergunta: Como é 

KeyboardInterrupt: ignored

In [ ]:
question = "Como se faz o armazenamento de peras?"
answer1 = response(modelo_classificador_geral, question)
print(answer1['Pergunta'])

Como se faz o armazenamento de peras?


In [ ]:
value = 0
maior_index = None
max_row =[] 

for index, row in df_cap_final.iterrows():
  #print(row['Pergunta'])
  valorsimilar = similar(  question, row['Pergunta'])
  if (valorsimilar > value):
    maior_index = index
    print(index)
    max_row = row
    value = valorsimilar
    
    #print(value)

print(value)
print(max_row)
print(index)

402
403
404
406
0.6666666666666666
Numero                                                         407
Livro                                                         Pera
Capitulo                    Colheita, Pós-colheita e Processamento
Pergunta                        Como é feita a colheita das peras?
Resposta         A colheita das peras é realizada manualmente, ...
Target                                                        Pera
Target_final                                                Frutas
Target_final1                                                  NaN
Target_final2                                               Frutas
Target_final3                                                  NaN
Name: 406, dtype: object
481


In [ ]:
numbers = [55, 4, 92, 1, 104, 64, 73, 99, 20]

max_value = None

for num in numbers:
    if (max_value is None or num > max_value):
        max_value = num

print('Maximum value:', max_value)

Maximum value: 104
